In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
import random

matches = pd.read_csv("matches.csv", index_col=0)

# Convert the "date" column to datetime format
matches["date"] = pd.to_datetime(matches["date"])

# Encode categorical features as numerical codes
matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["team_code"] = matches["team"].astype("category").cat.codes

# Extract additional features from existing columns
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
matches["day_code"] = matches["date"].dt.dayofweek

# Map the "result" column to numeric values
result_mapping = {'W': 1, 'D': 0, 'L': 2}
matches['result_numeric'] = matches["result"].map(result_mapping)

# Remove insignificant columns
insignificant_columns = ["match report", "notes", "tkl.1", "tkl.2", "tkl.3"]
#matches = matches.drop(insignificant_columns, axis=1)

additional_insignificant_columns = ["referee", "captain", "formation"]
matches = matches.drop(additional_insignificant_columns, axis=1)

matches = matches.sort_values(by='date', ascending=True)
matches.index = range(matches.shape[0])

cols_to_replace = ["xg", "xga", "sh", "sot", "dist", "fk", "pk", "pkatt", "sca", "gca", "att", "cmp", "poss_y", "def pen",
                   "def 3rd","mid 3rd", "att 3rd", "att pen", "prgc", "1/3_x", "cpa", "tkl", "blocks", "int", "kp", "1/3_y",
                   "prgp","saves", "crdy","crdr", "fls", "fld", "off"]

# Identify the rows for Matchweek 38 of the 2023 season
mask = (matches['round'] == 'Matchweek 38') & (matches['season'] == 2023)

# For each team, calculate the mean of the columns to replace for the 2023 season
team_means = matches[matches['season'] == 2023].groupby('team')[cols_to_replace].mean()

# For each row of Matchweek 38, replace the columns with the team's mean
for idx, row in matches[mask].iterrows():
    team = row['team']
    matches.loc[idx, cols_to_replace] = team_means.loc[team]
    
matches 



,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fld,off,season,team,venue_code,opp_code,team_code,hour,day_code,result_numeric
0,2021-08-13,20:00,Premier League,Matchweek 1,Fri,Home,W,2.0,0.0,Arsenal,...,7.000000,1.000000,2022,Brentford,1,0,3,20,4,1
1,2021-08-13,20:00,Premier League,Matchweek 1,Fri,Away,L,0.0,2.0,Brentford,...,12.000000,1.000000,2022,Arsenal,0,3,0,20,4,2
2,2021-08-14,15:00,Premier League,Matchweek 1,Sat,Home,L,1.0,2.0,Brighton,...,7.000000,1.000000,2022,Burnley,1,4,5,15,5,2
3,2021-08-14,15:00,Premier League,Matchweek 1,Sat,Home,W,1.0,0.0,Wolves,...,10.000000,5.000000,2022,Leicester City,1,22,11,15,5,1
4,2021-08-14,15:00,Premier League,Matchweek 1,Sat,Away,L,0.0,3.0,Chelsea,...,14.000000,1.000000,2022,Crystal Palace,0,6,7,15,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,2023-05-28,16:30,Premier League,Matchweek 38,Sun,Away,L,0.0,1.0,Brentford,...,10.105263,1.342105,2023,Manchester City,0,3,13,16,6,2
1516,2023-05-28,16:30,Premier League,Matchweek 38,Sun,Home,D,1.0,1.0,Newcastle Utd,...,12.157895,1.947368,2023,Chelsea,1,15,6,16,6,0
1517,2023-05-28,16:30,Premier League,Matchweek 38,Sun,Home,W,1.0,0.0,Manchester City,...,10.500000,1.894737,2023,Brentford,1,13,3,16,6,1
1518,2023-05-28,16:30,Premier League,Matchweek 38,Sun,Away,D,4.0,4.0,Southampton,...,8.447368,2.210526,2023,Liverpool,0,18,12,16,6,0


In [3]:
initial_predictors = ["xg", "xga", "sh", "sot", "dist", "fk", "pk", "pkatt", "sca", "gca", "att", "cmp", "poss_y", "def pen",
              "def 3rd","mid 3rd", "att 3rd", "att pen", "prgc", "1/3_x", "cpa", "tkl", "blocks", "int", "kp", "1/3_y",
              "prgp","saves", "crdy","crdr", "fls", "fld", "off", "venue_code","opp_code", "team_code"]

date_dict = {
    'Entry1': ['2021-08-01', '2023-01-06'],
    'Entry2': ['2022-08-01', '2023-04-17'],
    'Entry3': ['2023-03-16', '2023-05-19']
}

param_grid_rfc = {
    'bootstrap': [True, False],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [20, 40, 60, 80, 100, 200, 400, 600, 800, 1000]
}

param_grid_svm = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'class_weight': ['balanced', None],
    'degree': [2, 3, 4]
}

param_grid_knn = {
    'n_neighbors': list(range(1,31)),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

param_grid_xgb = {
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': [3, 4, 5, 6, 7],
    'min_child_weight': [1, 2, 3, 4],
    'subsample': [0.5, 0.7, 0.9, 1],
    'n_estimators': [100, 200, 300, 400, 500]
}

team_name_mapping = {
    "Manchester Utd": "Manchester United",
    "Newcastle Utd" : "Newcastle United",
    "Nott'ham Forest": "Nottingham Forest",
    "Brighton and Hove Albion" : "Brighton",
    "West Ham United": "West Ham",
    "Tottenham Hotspur": "Tottenham",
    "Wolverhampton Wanderers": "Wolves"
    # Add more mappings here if needed
}


////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////
RANDOM FOREST CLASSIFIER
//////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////

In [3]:
results = {}

for key, value in date_dict.items():
    if key == 'Entry1':
        print("Results for 2 seasons")
    elif key == 'Entry2':
        print("Results for 1 season")
    elif key == 'Entry3':
        print("Results for recent matches")
    train = matches[(matches["date"] > value[0]) & (matches["date"] < value[1])]
    test = matches[matches["date"] >= value[1]]

    X_train = train[initial_predictors]
    y_train = train['result_numeric']
    X_test = test[initial_predictors]
    y_test = test['result_numeric']

    # Define the model
    model = RandomForestClassifier()
    
    rf_random = RandomizedSearchCV(estimator=model, param_distributions=param_grid_rfc, 
                               n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)
    # Fit the random search model
    rf_random.fit(X_train, y_train)
    
    y_pred = rf_random.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(accuracy_score(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    
    print(rf_random.best_params_)
    
    correlation_matrix = train[initial_predictors + ['result_numeric']].corr()
    correlation_with_target = correlation_matrix['result_numeric'].abs().sort_values(ascending=False)
    
    estimator = LogisticRegressionCV(penalty='l1', solver='liblinear', cv=5, max_iter=1000)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    rfe = RFE(estimator, n_features_to_select=10)
    rfe.fit(X_train_scaled, y_train)
    selected_features_rfe = X_train.columns[rfe.support_]
    
    print("\nCorrelation with Target:")
    print(correlation_with_target)
    print("\nSelected Features using RFE:")
    print(selected_features_rfe)
    
    best_params = rf_random.best_params_
    selected_features = selected_features_rfe
    top_correlations = correlation_with_target.index[1:11].tolist()
    dates = [value[0],value[1]]
    results[accuracy] = {'parameters': best_params, 'features': selected_features,
                         'correlations': top_correlations, 'dates': dates}

Results for 2 seasons
Fitting 3 folds for each of 100 candidates, totalling 300 fits
0.6495098039215687
[[ 16  31  45]
 [ 18 118  22]
 [ 13  14 131]]
              precision    recall  f1-score   support

           0       0.34      0.17      0.23        92
           1       0.72      0.75      0.74       158
           2       0.66      0.83      0.74       158

    accuracy                           0.65       408
   macro avg       0.58      0.58      0.57       408
weighted avg       0.61      0.65      0.62       408

{'n_estimators': 20, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 40, 'bootstrap': True}

Correlation with Target:
result_numeric    1.000000
xga               0.351718
gca               0.231967
sh                0.204843
xg                0.203812
sca               0.199476
kp                0.195963
sot               0.188528
att pen           0.173191
cpa               0.163143
att 3rd           0.130033
prgc              

In [4]:
max_accuracy = max(results.keys())

# Retrieve the best parameters, features, and correlations associated with the highest accuracy
best_params = results[max_accuracy]['parameters']
best_features = results[max_accuracy]['features']
best_correlations = results[max_accuracy]['correlations']
best_time = results[max_accuracy]['dates']
selected_features = {
    'all_features': initial_predictors,
    'best_features': best_features,
    'best_correlations': best_correlations
}

rf_best = RandomForestClassifier(n_estimators=best_params['n_estimators'], 
                                 min_samples_split=best_params['min_samples_split'], 
                                 min_samples_leaf=best_params['min_samples_leaf'], 
                                 max_features=best_params['max_features'], 
                                 max_depth=best_params['max_depth'], 
                                 bootstrap=best_params['bootstrap'],
                                 class_weight='balanced')

for subset_name, predictors in selected_features.items():
    print(f"Testing feature subset: {subset_name}")
    print(f"for features: {predictors}")
    # Split the data into training and testing sets
    train = matches[(matches["date"] > best_time[0]) & (matches["date"] < best_time[1])]
    test = matches[matches["date"] >= best_time[1]]
    X_train = train[predictors]
    y_train = train['result_numeric']
    X_test = test[predictors]
    y_test = test['result_numeric']

    # Fit the model using the rf_best
    rf_best.fit(X_train, y_train)
    
    # Predict and evaluate
    y_pred = rf_best.predict(X_test)
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"Confusion Matrix: \n {confusion_matrix(y_test, y_pred)}")
    print(f"Classification Report: \n {classification_report(y_test, y_pred)}")
    print("\n")

Testing feature subset: all_features
for features: ['xg', 'xga', 'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt', 'sca', 'gca', 'att', 'cmp', 'poss_y', 'def pen', 'def 3rd', 'mid 3rd', 'att 3rd', 'att pen', 'prgc', '1/3_x', 'cpa', 'tkl', 'blocks', 'int', 'kp', '1/3_y', 'prgp', 'saves', 'crdy', 'crdr', 'fls', 'fld', 'off', 'venue_code', 'opp_code', 'team_code']
Accuracy: 0.68
Confusion Matrix: 
 [[ 9 15  8]
 [ 7 46  6]
 [ 5  7 47]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.43      0.28      0.34        32
           1       0.68      0.78      0.72        59
           2       0.77      0.80      0.78        59

    accuracy                           0.68       150
   macro avg       0.63      0.62      0.62       150
weighted avg       0.66      0.68      0.67       150



Testing feature subset: best_features
for features: Index(['xga', 'sh', 'sca', 'gca', 'att', 'cmp', 'def pen', 'def 3rd',
       'mid 3rd', 'att 3rd'],
      dtype=

////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////
SUPPORT VECTOR MACHINE
//////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////

In [5]:
results = {}

for key, value in date_dict.items():
    if key == 'Entry1':
        print("Results for 2 seasons")
    elif key == 'Entry2':
        print("Results for 1 season")
    elif key == 'Entry3':
        print("Results for recent matches")
    train = matches[(matches["date"] > value[0]) & (matches["date"] < value[1])]
    test = matches[matches["date"] >= value[1]]

    X_train = train[initial_predictors]
    y_train = train['result_numeric']
    X_test = test[initial_predictors]
    y_test = test['result_numeric']
    
    model = SVC()
    
    svm_random = RandomizedSearchCV(estimator=model, param_distributions=param_grid_svm, 
                                n_iter=20, cv=2, verbose=2, random_state=42, n_jobs=-1)

    svm_random.fit(X_train, y_train)
    
    y_pred = svm_random.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(accuracy_score(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    
    correlation_matrix = train[initial_predictors + ['result_numeric']].corr()
    correlation_with_target = correlation_matrix['result_numeric'].abs().sort_values(ascending=False)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    rfe = RFE(estimator=SVC(kernel='linear'), n_features_to_select=10, step=1)
    rfe.fit(X_train_scaled, y_train)
    selected_features_rfe = X_train.columns[rfe.support_]
    
    print("\nCorrelation with Target:")
    print(correlation_with_target)
    print("\nSelected Features using RFE:")
    print(selected_features_rfe)
    
    best_params = svm_random.best_params_
    selected_features = selected_features_rfe
    top_correlations = correlation_with_target.index[1:11].tolist()
    dates = [value[0],value[1]]
    results[accuracy] = {'parameters': best_params, 'features': selected_features,
                         'correlations': top_correlations, 'dates': dates}

Results for 2 seasons
Fitting 2 folds for each of 20 candidates, totalling 40 fits
0.6691176470588235
[[ 30  23  39]
 [ 28 117  13]
 [ 24   8 126]]
              precision    recall  f1-score   support

           0       0.37      0.33      0.34        92
           1       0.79      0.74      0.76       158
           2       0.71      0.80      0.75       158

    accuracy                           0.67       408
   macro avg       0.62      0.62      0.62       408
weighted avg       0.66      0.67      0.66       408


Correlation with Target:
result_numeric    1.000000
xga               0.351718
gca               0.231967
sh                0.204843
xg                0.203812
sca               0.199476
kp                0.195963
sot               0.188528
att pen           0.173191
cpa               0.163143
att 3rd           0.130033
prgc              0.129672
prgp              0.123288
poss_y            0.104324
crdr              0.098819
saves             0.094050
1/3_y        

In [6]:
max_accuracy = max(results.keys())

# Retrieve the best parameters, features, and correlations associated with the highest accuracy
best_params = results[max_accuracy]['parameters']
best_features = results[max_accuracy]['features']
best_correlations = results[max_accuracy]['correlations']
best_time = results[max_accuracy]['dates']
selected_features = {
    'all_features': initial_predictors,
    'best_features': best_features,
    'best_correlations': best_correlations
}

svm_best = SVC(kernel= best_params['kernel'],
              gamma= best_params['gamma'],
              degree= best_params['degree'],
              class_weight= best_params['class_weight'],
              C= best_params['C'])

for subset_name, predictors in selected_features.items():
    print(f"Testing feature subset: {subset_name}")
    print(f"for features: {predictors}")
    # Split the data into training and testing sets
    train = matches[(matches["date"] > best_time[0]) & (matches["date"] < best_time[1])]
    test = matches[matches["date"] >= best_time[1]]
    X_train = train[predictors]
    y_train = train['result_numeric']
    X_test = test[predictors]
    y_test = test['result_numeric']
    
    # Scale the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Fit the model
    svm_best.fit(X_train_scaled, y_train)
    
    # Predict and evaluate
    y_pred = svm_best.predict(X_test_scaled)
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"Confusion Matrix: \n {confusion_matrix(y_test, y_pred)}")
    print(f"Classification Report: \n {classification_report(y_test, y_pred)}")
    print("\n")

Testing feature subset: all_features
for features: ['xg', 'xga', 'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt', 'sca', 'gca', 'att', 'cmp', 'poss_y', 'def pen', 'def 3rd', 'mid 3rd', 'att 3rd', 'att pen', 'prgc', '1/3_x', 'cpa', 'tkl', 'blocks', 'int', 'kp', '1/3_y', 'prgp', 'saves', 'crdy', 'crdr', 'fls', 'fld', 'off', 'venue_code', 'opp_code', 'team_code']
Accuracy: 0.72
Confusion Matrix: 
 [[13 10  9]
 [ 6 47  6]
 [ 7  4 48]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.50      0.41      0.45        32
           1       0.77      0.80      0.78        59
           2       0.76      0.81      0.79        59

    accuracy                           0.72       150
   macro avg       0.68      0.67      0.67       150
weighted avg       0.71      0.72      0.71       150



Testing feature subset: best_features
for features: Index(['xga', 'sh', 'sca', 'gca', 'att', 'cmp', 'def pen', 'mid 3rd',
       'att 3rd', '1/3_y'],
      dtype='o

////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////
K - NEAREST NEIGHBOURS
//////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////

In [7]:
results = {}

for key, value in date_dict.items():
    if key == 'Entry1':
        print("Results for 2 seasons")
    elif key == 'Entry2':
        print("Results for 1 season")
    elif key == 'Entry3':
        print("Results for recent matches")
        
    scaler = StandardScaler()
    
    train = matches[(matches["date"] > value[0]) & (matches["date"] < value[1])]
    test = matches[matches["date"] >= value[1]]

    X_train = train[initial_predictors]
    y_train = train['result_numeric']
    X_test = test[initial_predictors]
    y_test = test['result_numeric']
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    col_names = X_train.columns
    
    model = KNeighborsClassifier()
    
    knn_random = RandomizedSearchCV(estimator=model, param_distributions=param_grid_knn, 
                               n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

    knn_random.fit(X_train_scaled, y_train)
    
    y_pred = knn_random.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    print(accuracy_score(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    
    correlation_matrix = train[initial_predictors + ['result_numeric']].corr()
    correlation_with_target = correlation_matrix['result_numeric'].abs().sort_values(ascending=False)
    
    print(knn_random.best_params_)
    X_train = pd.DataFrame(X_train_scaled, columns=col_names)
    X_test = pd.DataFrame(X_test_scaled, columns=col_names)
    knn_random.best_estimator_.fit(X_train_scaled, y_train)
    
    estimator = LogisticRegressionCV(penalty='l1', solver='liblinear', cv=5, max_iter=1000)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_scaled)
    X_test_scaled = scaler.transform(X_test_scaled)
    rfe = RFE(estimator, n_features_to_select=8)
    rfe.fit(X_train_scaled, y_train)
    
    print("\nCorrelation with Target:")
    print(correlation_with_target)
    print("\nSelected Features using RFE:")
    print(selected_features_rfe)
    
    best_params = knn_random.best_params_
    selected_features = selected_features_rfe
    top_correlations = correlation_with_target.index[1:11].tolist()
    dates = [value[0],value[1]]
    results[accuracy] = {'parameters': best_params, 'features': selected_features,
                         'correlations': top_correlations, 'dates': dates}

Results for 2 seasons
Fitting 3 folds for each of 100 candidates, totalling 300 fits
0.6151960784313726
[[  9  40  43]
 [ 19 117  22]
 [  9  24 125]]
              precision    recall  f1-score   support

           0       0.24      0.10      0.14        92
           1       0.65      0.74      0.69       158
           2       0.66      0.79      0.72       158

    accuracy                           0.62       408
   macro avg       0.52      0.54      0.52       408
weighted avg       0.56      0.62      0.58       408

{'weights': 'distance', 'n_neighbors': 28, 'metric': 'manhattan'}

Correlation with Target:
result_numeric    1.000000
xga               0.351718
gca               0.231967
sh                0.204843
xg                0.203812
sca               0.199476
kp                0.195963
sot               0.188528
att pen           0.173191
cpa               0.163143
att 3rd           0.130033
prgc              0.129672
prgp              0.123288
poss_y            0.104324

In [8]:
max_accuracy = max(results.keys())

# Retrieve the best parameters, features, and correlations associated with the highest accuracy
best_params = results[max_accuracy]['parameters']
best_features = results[max_accuracy]['features']
best_correlations = results[max_accuracy]['correlations']
best_time = results[max_accuracy]['dates']
print(best_time)
selected_features = {
    'all_features': initial_predictors,
    'best_features': best_features,
    'best_correlations': best_correlations
}

knn_best = KNeighborsClassifier(n_neighbors= best_params['n_neighbors'],
                                weights= best_params['weights'],
                                metric= best_params['metric'])
        
for subset_name, predictors in selected_features.items():
    print(f"Testing feature subset: {subset_name}")
    print(f"for features: {predictors}")
    # Split the data into training and testing sets
    train = matches[(matches["date"] > best_time[0]) & (matches["date"] < best_time[1])]
    test = matches[matches["date"] >= best_time[1]]
    X_train = train[predictors]
    y_train = train['result_numeric']
    X_test = test[predictors]
    y_test = test['result_numeric']
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    col_names = X_train.columns
    # Fit the model using the RandomizedSearchCV, same as before
    knn_best.fit(X_train_scaled, y_train)
    
    # Predict and evaluate
    y_pred = knn_best.predict(X_test_scaled)
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"Confusion Matrix: \n {confusion_matrix(y_test, y_pred)}")
    print(f"Classification Report: \n {classification_report(y_test, y_pred)}")
    print("\n")                

['2022-08-01', '2023-04-17']
Testing feature subset: all_features
for features: ['xg', 'xga', 'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt', 'sca', 'gca', 'att', 'cmp', 'poss_y', 'def pen', 'def 3rd', 'mid 3rd', 'att 3rd', 'att pen', 'prgc', '1/3_x', 'cpa', 'tkl', 'blocks', 'int', 'kp', '1/3_y', 'prgp', 'saves', 'crdy', 'crdr', 'fls', 'fld', 'off', 'venue_code', 'opp_code', 'team_code']
Accuracy: 0.6266666666666667
Confusion Matrix: 
 [[ 1 17 14]
 [ 3 49  7]
 [ 0 15 44]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.25      0.03      0.06        32
           1       0.60      0.83      0.70        59
           2       0.68      0.75      0.71        59

    accuracy                           0.63       150
   macro avg       0.51      0.54      0.49       150
weighted avg       0.56      0.63      0.57       150



Testing feature subset: best_features
for features: Index(['xga', 'pk', 'pkatt', 'sca', 'gca', 'cmp', 'poss_y', 'kp', 'sa

////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////
EXTREME GRADIENT BOOSTING
//////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////////

In [13]:
results = {}

for key, value in date_dict.items():
    if key == 'Entry1':
        print("Results for 2 seasons")
    elif key == 'Entry2':
        print("Results for 1 season")
    elif key == 'Entry3':
        print("Results for recent matches")
    train = matches[(matches["date"] > value[0]) & (matches["date"] < value[1])]
    test = matches[matches["date"] >= value[1]]

    X_train = train[initial_predictors]
    y_train = train['result_numeric']
    X_test = test[initial_predictors]
    y_test = test['result_numeric']
    
    model = xgb.XGBClassifier(use_label_encoder=False)
    
    xgb_random = RandomizedSearchCV(estimator=model, param_distributions=param_grid_xgb, 
                                n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

    xgb_random.fit(X_train, y_train)
    
    y_pred = xgb_random.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(accuracy_score(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    
    correlation_matrix = train[initial_predictors + ['result_numeric']].corr()
    correlation_with_target = correlation_matrix['result_numeric'].abs().sort_values(ascending=False)
    
    scaler = StandardScaler()
    best_xgb = xgb_random.best_estimator_
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    rfe = RFE(estimator=best_xgb, n_features_to_select=8)
    rfe.fit(X_train_scaled, y_train)
    selected_features_rfe = X_train.columns[rfe.support_]
    
    print("\nCorrelation with Target:")
    print(correlation_with_target)
    print("\nSelected Features using RFE:")
    print(selected_features_rfe)
    
    best_params = xgb_random.best_params_
    selected_features = selected_features_rfe
    top_correlations = correlation_with_target.index[1:11].tolist()
    dates = [value[0],value[1]]
    results[accuracy] = {'parameters': best_params, 'features': selected_features,
                         'correlations': top_correlations, 'dates': dates}

Results for 2 seasons
Fitting 3 folds for each of 100 candidates, totalling 300 fits
0.6862745098039216
[[ 23  25  44]
 [ 18 122  18]
 [ 13  10 135]]
              precision    recall  f1-score   support

           0       0.43      0.25      0.32        92
           1       0.78      0.77      0.77       158
           2       0.69      0.85      0.76       158

    accuracy                           0.69       408
   macro avg       0.63      0.63      0.62       408
weighted avg       0.66      0.69      0.67       408


Correlation with Target:
result_numeric    1.000000
xga               0.351718
gca               0.231967
sh                0.204843
xg                0.203812
sca               0.199476
kp                0.195963
sot               0.188528
att pen           0.173191
cpa               0.163143
att 3rd           0.130033
prgc              0.129672
prgp              0.123288
poss_y            0.104324
crdr              0.098819
saves             0.094050
1/3_y      

In [10]:
max_accuracy = max(results.keys())

# Retrieve the best parameters, features, and correlations associated with the highest accuracy
best_params = results[max_accuracy]['parameters']
best_features = results[max_accuracy]['features']
best_correlations = results[max_accuracy]['correlations']
best_time = results[max_accuracy]['dates']
selected_features = {
    'all_features': initial_predictors,
    'best_features': best_features,
    'best_correlations': best_correlations
}

xgb_best = xgb.XGBClassifier(
    learning_rate = best_params['learning_rate'],
    max_depth = best_params['max_depth'],
    min_child_weight = best_params['min_child_weight'],
    subsample = best_params['subsample'],
    n_estimators =  best_params['n_estimators']
)

for subset_name, predictors in selected_features.items():
    print(f"Testing feature subset: {subset_name}")
    print(f"for features: {predictors}")
    # Split the data into training and testing sets
    train = matches[(matches["date"] > best_time[0]) & (matches["date"] < best_time[1])]
    test = matches[matches["date"] >= best_time[1]]
    X_train = train[predictors]
    y_train = train['result_numeric']
    X_test = test[predictors]
    y_test = test['result_numeric']
    
     # Fit the model using the rf_best
    xgb_best.fit(X_train, y_train)
    
    # Predict and evaluate
    y_pred = xgb_best.predict(X_test)
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"Confusion Matrix: \n {confusion_matrix(y_test, y_pred)}")
    print(f"Classification Report: \n {classification_report(y_test, y_pred)}")
    print("\n")

Testing feature subset: all_features
for features: ['xg', 'xga', 'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt', 'sca', 'gca', 'att', 'cmp', 'poss_y', 'def pen', 'def 3rd', 'mid 3rd', 'att 3rd', 'att pen', 'prgc', '1/3_x', 'cpa', 'tkl', 'blocks', 'int', 'kp', '1/3_y', 'prgp', 'saves', 'crdy', 'crdr', 'fls', 'fld', 'off', 'venue_code', 'opp_code', 'team_code']
Accuracy: 0.6866666666666666
Confusion Matrix: 
 [[ 2 12 18]
 [ 3 48  8]
 [ 1  5 53]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.33      0.06      0.11        32
           1       0.74      0.81      0.77        59
           2       0.67      0.90      0.77        59

    accuracy                           0.69       150
   macro avg       0.58      0.59      0.55       150
weighted avg       0.63      0.69      0.63       150



Testing feature subset: best_features
for features: Index(['xga', 'gca', 'att', 'cmp', 'blocks', 'saves', 'crdr', 'fls'], dtype='object')
Accuracy: 0.

//////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// ////////////////////////////////////////////////////////////////////////// RFC SIMULATION FOR 1X2 BET ////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// ////////////////////////////////////////////////////////////////////////////////////////////////////

In [14]:
hyperparameters = {'n_estimators': 80, 'min_samples_split': 2, 
                   'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 30, 'bootstrap': True}

matches2 = matches
matches2["pred0"] = 0
matches2["pred1"] = 0
matches2["pred2"] = 0
rf_last = matches2.iloc[-20:][["team", "opponent","venue","result_numeric","pred0","pred1","pred2"]]
rf_last.reset_index(inplace=True)

for i in range(2500):
    features = ['xga', 'sh', 'sca', 'gca', 'att', 'cmp', 'def pen', 'def 3rd',
       'mid 3rd', 'att 3rd']
    train = matches[(matches["date"] < '2023-05-27') & (matches["date"] > '2022-08-01')]
    test = matches[matches["date"] >= '2023-05-27']
    X_train = train[features]
    y_train = train['result_numeric']
    X_test = test[features]
    y_test = test['result_numeric']
    seed = random.randint(0,10000000)
    # Fit the model using the rf_best
    rf_best = RandomForestClassifier(n_estimators=hyperparameters['n_estimators'], 
                                 min_samples_split=hyperparameters['min_samples_split'], 
                                 min_samples_leaf=hyperparameters['min_samples_leaf'], 
                                 max_features=hyperparameters['max_features'], 
                                 max_depth=hyperparameters['max_depth'], 
                                 bootstrap=hyperparameters['bootstrap'],
                                 class_weight='balanced',
                                 random_state=random.seed(seed)   
                                    )
    rf_best.fit(X_train, y_train)
    # Predict and evaluate
    y_pred = rf_best.predict(X_test)
#     print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
#     print(f"Confusion Matrix: \n {confusion_matrix(y_test, y_pred)}")
#     print(f"Classification Report: \n {classification_report(y_test, y_pred)}")
#     print("\n")
    #print(y_pred)
    if i%250==0:
        print(i, seed)
    for i in range(len(y_pred)):
        if y_pred[i] == 0:
            rf_last.loc[i,"pred0"] += 1
        elif y_pred[i] == 1:
            rf_last.loc[i,"pred1"] += 1
        elif y_pred[i] == 2:
            rf_last.loc[i,"pred2"] += 1

0 6602515
250 8749374
500 5938031
750 7795275
1000 569007
1250 3134201
1500 9470078
1750 2992305
2000 3745108
2250 8641197


In [15]:
# Apply the mapping to the 'team' and 'opponent' columns
rf_last['team'] = rf_last['team'].replace(team_name_mapping)
rf_last['opponent'] = rf_last['opponent'].replace(team_name_mapping)

# Now, we can proceed with the merging process as before:

def create_match_id(row):
    return ''.join(sorted([row['team'], row['opponent']]))

# Add match_id to the DataFrame
rf_last['match_id'] = rf_last.apply(create_match_id, axis=1)

# Split the DataFrame into Home and Away
rf_last_home = rf_last[rf_last['venue'] == 'Home'].copy()
rf_last_away = rf_last[rf_last['venue'] == 'Away'].copy()

# Merge the Home and Away DataFrames on match_id
rf_last_merged = pd.merge(rf_last_home, rf_last_away, on='match_id', suffixes=('_home', '_away'))

# Perform the desired operations on the pred columns
rf_last_merged['pred0'] = rf_last_merged['pred0_home'] + rf_last_merged['pred0_away']
rf_last_merged['pred1'] = rf_last_merged['pred1_home'] + rf_last_merged['pred2_away']
rf_last_merged['pred2'] = rf_last_merged['pred2_home'] + rf_last_merged['pred1_away']

# Clean up the DataFrame
rf_last_final = rf_last_merged[['team_home', 'opponent_home', 'result_numeric_home', 'pred0', 'pred1', 'pred2']]
rf_last_final = rf_last_final.rename(columns={
    'team_home': 'team',
    'opponent_home': 'opponent',
    'result_numeric_home': 'result_numeric'
})
print(rf_last_final)

                team           opponent  result_numeric  pred0  pred1  pred2
0     Crystal Palace  Nottingham Forest               0   3838   1096     66
1       Leeds United          Tottenham               2   1016      0   3984
2            Arsenal             Wolves               1    568   3997    435
3     Leicester City           West Ham               1   3440      3   1557
4  Manchester United             Fulham               1   1345   3655      0
5            Everton        Bournemouth               1   3336    457   1207
6        Aston Villa           Brighton               1    890   1911   2199
7            Chelsea   Newcastle United               0   2461     37   2502
8          Brentford    Manchester City               1      0   2500   2500
9        Southampton          Liverpool               0   3538      7   1455


//////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// ////////////////////////////////////////////////////////////////////////// SVM SIMULATION FOR 1X2 BET ////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// ////////////////////////////////////////////////////////////////////////////////////////////////////

In [4]:
hyperparameters =  {'kernel': 'linear', 'gamma': 1, 'degree': 3, 'class_weight': None, 'C': 1}

matches3 = matches
matches3["pred0"] = 0
matches3["pred1"] = 0
matches3["pred2"] = 0
svm_last = matches3.iloc[-20:][["team", "opponent","venue","result_numeric","pred0","pred1","pred2"]]
svm_last.reset_index(inplace=True)

for i in range(150):
    features = ['xga', 'sh', 'sca', 'gca', 'att', 'cmp', 'def pen', 'mid 3rd',
       'att 3rd', '1/3_y']

    # Split the data into training and testing sets
    train = matches[(matches["date"] < '2023-05-27') & (matches["date"] > '2022-08-01')]
    test = matches[matches["date"] >= '2023-05-27']
    X_train = train[features]
    y_train = train['result_numeric']
    X_test = test[features]
    y_test = test['result_numeric']
    seed = random.randint(0,10000000)
    # Define the model with the best parameters
    svm_best = SVC(kernel = hyperparameters["kernel"],
                  gamma = hyperparameters['gamma'],
                  degree = hyperparameters['degree'],
                  class_weight = hyperparameters['class_weight'],
                  C = hyperparameters['C'],
                  random_state = seed)

    # Fit the model
    svm_best.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = svm_best.predict(X_test)
#     print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
#     print(f"Confusion Matrix: \n {confusion_matrix(y_test, y_pred)}")
#     print(f"Classification Report: \n {classification_report(y_test, y_pred)}")
#     print("\n")
    if i == 2:
        print("simulation started")
    if i%50==0:
        print(i, seed)
    for i in range(len(y_pred)):
        if y_pred[i] == 0:
            svm_last.loc[i,"pred0"] += 1
        elif y_pred[i] == 1:
            svm_last.loc[i,"pred1"] += 1
        elif y_pred[i] == 2:
            svm_last.loc[i,"pred2"] += 1
        

0 8714813
simulation started
50 6836857
100 9961995


In [7]:
# Apply the mapping to the 'team' and 'opponent' columns
svm_last['team'] = svm_last['team'].replace(team_name_mapping)
svm_last['opponent'] = svm_last['opponent'].replace(team_name_mapping)

# Now, we can proceed with the merging process as before:

def create_match_id(row):
    return ''.join(sorted([row['team'], row['opponent']]))

# Add match_id to the DataFrame
svm_last['match_id'] = svm_last.apply(create_match_id, axis=1)

# Split the DataFrame into Home and Away
svm_last_home = svm_last[svm_last['venue'] == 'Home'].copy()
svm_last_away = svm_last[svm_last['venue'] == 'Away'].copy()

# Merge the Home and Away DataFrames on match_id
svm_last_merged = pd.merge(svm_last_home, svm_last_away, on='match_id', suffixes=('_home', '_away'))

# Perform the desired operations on the pred columns
svm_last_merged['pred0'] = svm_last_merged['pred0_home'] + svm_last_merged['pred0_away']
svm_last_merged['pred1'] = svm_last_merged['pred1_home'] + svm_last_merged['pred2_away']
svm_last_merged['pred2'] = svm_last_merged['pred2_home'] + svm_last_merged['pred1_away']

# Clean up the DataFrame
svm_last_final = svm_last_merged[['team_home', 'opponent_home', 'result_numeric_home', 'pred0', 'pred1', 'pred2']]
svm_last_final = svm_last_final.rename(columns={
    'team_home': 'team',
    'opponent_home': 'opponent',
    'result_numeric_home': 'result_numeric'
})
print(svm_last_final)

                team           opponent  result_numeric  pred0  pred1  pred2
0     Crystal Palace  Nottingham Forest               0    150    150      0
1       Leeds United          Tottenham               2      0      0    300
2            Arsenal             Wolves               1      0    300      0
3     Leicester City           West Ham               1    150      0    150
4  Manchester United             Fulham               1    150    150      0
5            Everton        Bournemouth               1      0    150    150
6        Aston Villa           Brighton               1      0    150    150
7            Chelsea   Newcastle United               0      0      0    300
8          Brentford    Manchester City               1      0    150    150
9        Southampton          Liverpool               0      0      0    300


//////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// ////////////////////////////////////////////////////////////////////////// KNN SIMULATION FOR 1X2 BET ////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// ////////////////////////////////////////////////////////////////////////////////////////////////////

In [10]:
hyperparameters = {'weights': 'distance', 'n_neighbors': 23, 'metric': 'euclidean'}

matches4 = matches
matches4["pred0"] = 0
matches4["pred1"] = 0
matches4["pred2"] = 0
knn_last = matches4.iloc[-20:][["team", "opponent","venue","result_numeric","pred0","pred1","pred2"]]
knn_last.reset_index(inplace=True)

for i in range(250):
    features = ['xga', 'gca', 'xg', 'sh', 'sca', 'kp', 'sot', 'att pen', 'cpa', 'att 3rd']

    # Split the data into training and testing sets
    train = matches[(matches["date"] < '2023-05-27') & (matches["date"] > '2022-08-01')]
    test = matches[matches["date"] >= '2023-05-27']
    X_train = train[features]
    y_train = train['result_numeric']
    X_test = test[features]
    y_test = test['result_numeric']
    seed = random.randint(0,10000000)
    # Define the model with the best parameters
    knn_best = KNeighborsClassifier(weights = hyperparameters['weights'],
                                    n_neighbors = hyperparameters['n_neighbors'],
                                    metric = hyperparameters['metric']
                                   )

    # Fit the model
    knn_best.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = knn_best.predict(X_test)
#     print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
#     print(f"Confusion Matrix: \n {confusion_matrix(y_test, y_pred)}")
#     print(f"Classification Report: \n {classification_report(y_test, y_pred)}")
#     print("\n")
    if i%250==0:
        print(i, seed)
    for i in range(len(y_pred)):
        if y_pred[i] == 0:
            knn_last.loc[i,"pred0"] += 1
        elif y_pred[i] == 1:
            knn_last.loc[i,"pred1"] += 1
        elif y_pred[i] == 2:
            knn_last.loc[i,"pred2"] += 1
        

0 4749876


In [11]:
# Apply the mapping to the 'team' and 'opponent' columns
knn_last['team'] = knn_last['team'].replace(team_name_mapping)
knn_last['opponent'] = knn_last['opponent'].replace(team_name_mapping)

# Now, we can proceed with the merging process as before:

def create_match_id(row):
    return ''.join(sorted([row['team'], row['opponent']]))

# Add match_id to the DataFrame
knn_last['match_id'] = knn_last.apply(create_match_id, axis=1)

# Split the DataFrame into Home and Away
knn_last_home = knn_last[knn_last['venue'] == 'Home'].copy()
knn_last_away = knn_last[knn_last['venue'] == 'Away'].copy()

# Merge the Home and Away DataFrames on match_id
knn_last_merged = pd.merge(knn_last_home, knn_last_away, on='match_id', suffixes=('_home', '_away'))

# Perform the desired operations on the pred columns
knn_last_merged['pred0'] = knn_last_merged['pred0_home'] + knn_last_merged['pred0_away']
knn_last_merged['pred1'] = knn_last_merged['pred1_home'] + knn_last_merged['pred2_away']
knn_last_merged['pred2'] = knn_last_merged['pred2_home'] + knn_last_merged['pred1_away']

# Clean up the DataFrame
knn_last_final = knn_last_merged[['team_home', 'opponent_home', 'result_numeric_home', 'pred0', 'pred1', 'pred2']]
knn_last_final = knn_last_final.rename(columns={
    'team_home': 'team',
    'opponent_home': 'opponent',
    'result_numeric_home': 'result_numeric'
})
print(knn_last_final)

                team           opponent  result_numeric  pred0  pred1  pred2
0     Crystal Palace  Nottingham Forest               0    250    250      0
1       Leeds United          Tottenham               2      0      0    500
2            Arsenal             Wolves               1      0    500      0
3     Leicester City           West Ham               1    250      0    250
4  Manchester United             Fulham               1      0    500      0
5            Everton        Bournemouth               1      0    250    250
6        Aston Villa           Brighton               1    250    250      0
7            Chelsea   Newcastle United               0      0    250    250
8          Brentford    Manchester City               1      0      0    500
9        Southampton          Liverpool               0      0    250    250


//////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// ////////////////////////////////////////////////////////////////////////// XGB SIMULATION FOR 1X2 BET ////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// //////////////////////////////////////////////////////////////////////////////////////////////////// ////////////////////////////////////////////////////////////////////////////////////////////////////

In [6]:
hyperparameters = {'subsample': 0.7, 'n_estimators': 500, 'min_child_weight': 4, 'max_depth': 5}

matches5 = matches
matches5["pred0"] = 0
matches5["pred1"] = 0
matches5["pred2"] = 0
xgb_last = matches5.iloc[-20:][["team", "opponent","venue","result_numeric","pred0","pred1","pred2"]]
xgb_last.reset_index(inplace=True)

for i in range(1000):
    features = ['xga', 'gca', 'att', 'cmp', 'blocks', 'saves', 'crdr', 'fls']

    # Split the data into training and testing sets
    train = matches[(matches["date"] < '2023-05-27') & (matches["date"] > '2022-08-01')]
    test = matches[matches["date"] >= '2023-05-27']
    X_train = train[features]
    y_train = train['result_numeric']
    X_test = test[features]
    y_test = test['result_numeric']
    seed = random.randint(0,10000000)
    # Define the model with the best parameters
    xgb_best =  xgb.XGBClassifier(subsample= hyperparameters['subsample'],
                                  n_estimators = hyperparameters['n_estimators'],
                                  min_child_weight = hyperparameters['min_child_weight'],
                                  max_depth = hyperparameters['max_depth'],
                                  random_state = seed)

    # Fit the model
    xgb_best.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = xgb_best.predict(X_test)
#     print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
#     print(f"Confusion Matrix: \n {confusion_matrix(y_test, y_pred)}")
#     print(f"Classification Report: \n {classification_report(y_test, y_pred)}")
#     print("\n")
    if i%100==0:
        print(i, seed)
    for i in range(len(y_pred)):
        if y_pred[i] == 0:
            xgb_last.loc[i,"pred0"] += 1
        elif y_pred[i] == 1:
            xgb_last.loc[i,"pred1"] += 1
        elif y_pred[i] == 2:
            xgb_last.loc[i,"pred2"] += 1
        
    

0 1026733
100 8007291
200 4821792
300 3432251
400 3857758
500 1875671
600 9261885
700 8577445
800 1048385
900 3795716


In [7]:
# Apply the mapping to the 'team' and 'opponent' columns
xgb_last['team'] = xgb_last['team'].replace(team_name_mapping)
xgb_last['opponent'] = xgb_last['opponent'].replace(team_name_mapping)

# Now, we can proceed with the merging process as before:

def create_match_id(row):
    return ''.join(sorted([row['team'], row['opponent']]))

# Add match_id to the DataFrame
xgb_last['match_id'] = xgb_last.apply(create_match_id, axis=1)

# Split the DataFrame into Home and Away
xgb_last_home = xgb_last[xgb_last['venue'] == 'Home'].copy()
xgb_last_away = xgb_last[xgb_last['venue'] == 'Away'].copy()

# Merge the Home and Away DataFrames on match_id
xgb_last_merged = pd.merge(xgb_last_home, xgb_last_away, on='match_id', suffixes=('_home', '_away'))

# Perform the desired operations on the pred columns
xgb_last_merged['pred0'] = xgb_last_merged['pred0_home'] + xgb_last_merged['pred0_away']
xgb_last_merged['pred1'] = xgb_last_merged['pred1_home'] + xgb_last_merged['pred2_away']
xgb_last_merged['pred2'] = xgb_last_merged['pred2_home'] + xgb_last_merged['pred1_away']

# Clean up the DataFrame
xgb_last_final = xgb_last_merged[['team_home', 'opponent_home', 'result_numeric_home', 'pred0', 'pred1', 'pred2']]
xgb_last_final = xgb_last_final.rename(columns={
    'team_home': 'team',
    'opponent_home': 'opponent',
    'result_numeric_home': 'result_numeric'
})
print(xgb_last_final)

                team           opponent  result_numeric  pred0  pred1  pred2
0     Crystal Palace  Nottingham Forest               0    976    992     32
1       Leeds United          Tottenham               2      0     53   1947
2            Arsenal             Wolves               1      0   1609    391
3     Leicester City           West Ham               1   1256      6    738
4  Manchester United             Fulham               1    159   1741    100
5            Everton        Bournemouth               1   2000      0      0
6        Aston Villa           Brighton               1    953      0   1047
7            Chelsea   Newcastle United               0   1000      0   1000
8          Brentford    Manchester City               1    891    109   1000
9        Southampton          Liverpool               0   1947      0     53
